In [25]:
import json
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [24]:
import json
# Read all the annotations
ann = json.loads(open('/gsc/pipelines/mavis/reference_files/ensembl69_hg19_annotations.json').read())

# For each gene, only keep the 'best_transcript' of  those with 'aliases' (hugo names)
lst = []
for g in ann['genes']:
    for transcript in g['transcripts']:
        if g['aliases'] and transcript['is_best_transcript']:
            lst.append((g['name'], g['aliases'], transcript['name'])) 
            
best_transcripts = [i[2] for i in lst]
# gene_trans_info = = [(g['name'], g['aliases'], transcript) for g in ann['genes'] for transcript in g['transcripts'] if g['aliases'] and transcript['is_best_transcript']]

### looked into exon regions Nina gave me

In [12]:
f = '/home/szong/projects/resource/ens69/gene_exon_ens69_nina.txt.sorted.merge'
df = pd.read_csv(f, sep='\t', header=None, low_memory=False)
df.columns = ['chrm', 'start', 'end']
df.head(2)

,chrm,start,end
0,1,11869,12227
1,1,12595,12721


In [13]:
df['length'] = df.end - df.start

In [14]:
df.head(2)

,chrm,start,end,length
0,1,11869,12227,358
1,1,12595,12721,126


In [15]:
df.length.sum()

124920796

### looked into this ticket: https://www.bcgsc.ca/jira/browse/UGNE-1726

Counts of bases for the 3 regions (not including annotations or parts of the genome not used in our pipeline)

regions	total bases	number of Ns	effective size

exonic	116,444,539	1	116,444,538

intronic	1,439,505,280	2,586,253	1,436,919,024

intergenic	1,545,854,885	234,433,262	1,311,421,623

Sum of effective sizes = 2864785185

Effective size used for bamStats (calculated years ago, using a possibly different method) = 2864785220


So we're off by 35 bases, but I'd say thats close enough.

In [83]:
116444539/2864785185

0.040646865813780034

### looked into NCBI CCDS

In [103]:
# ftp://ftp.ncbi.nih.gov/pub/CCDS/archive/Hs37.3/
f = '/home/szong/Downloads/CCDS.20110907.txt'
df2 = pd.read_csv(f, sep='\t')
df2.head(2)
df2.shape

,#chromosome,nc_accession,gene,gene_id,ccds_id,ccds_status,cds_strand,cds_from,cds_to,cds_locations,match_type
0,1,NC_000001.8,LINC00115,79854,CCDS1.1,Withdrawn,-,801942,802433,801942-802433,Identical
1,1,NC_000001.10,SAMD11,148398,CCDS2.2,Public,+,861321,879532,"861321-861392, 865534-865715, 866418-866468, 8...",Identical


(27787, 11)

In [67]:
df3 = df2[['#chromosome', 'cds_locations']]

In [68]:
df3.head(2)

,#chromosome,cds_locations
0,1,801942-802433
1,1,"861321-861392, 865534-865715, 866418-866468, 8..."


In [69]:
s = df3.cds_locations.str.split(',', expand=True).stack()
s[:3]

0  0     801942-802433
1  0     861321-861392
   1     865534-865715
dtype: object

In [70]:
idx = s.index.get_level_values(0)

In [85]:
df4 = df3.loc[idx,:]
df4['exon'] = list(s)

In [86]:
df4.drop('cds_locations', inplace=True, axis=1)

In [87]:
df4.shape
df4.head(2)

(287286, 2)

,#chromosome,exon
0,1,801942-802433
1,1,861321-861392


In [91]:
df4[['exon_start', 'exon_end']] = df4.exon.str.split('-', expand=True)
df4.drop('exon', inplace=True, axis=1)

In [92]:
df4.tail()

,#chromosome,exon_start,exon_end
27785,Y,16168463,16168738
27786,Y,16835028,16835148
27786,Y,16936067,16936252
27786,Y,16941609,16942398
27786,Y,16952292,16953141


In [94]:
df4['length'] = df4.exon_end.astype(int) - df4.exon_start.astype(int)

In [96]:
df4.head()
df4.length.sum()

,#chromosome,exon_start,exon_end,length
0,1,801942,802433,491
1,1,861321,861392,71
1,1,865534,865715,181
1,1,866418,866468,50
1,1,871151,871275,124


46925404

In [98]:
of = '/home/szong/Downloads/exons.tmp'
df4.to_csv(of, index=False, sep='\t')

In [101]:
f = '/home/szong/Downloads/exons.tmp.sorted.merge'
df = pd.read_csv(f, sep='\t', header=None)

df['len'] = df[2] - df[1]
df.head(2)

,0,1,2,len
0,1,69090,70007,917
1,1,367658,368596,938


In [102]:
df['len'].sum()

32939631

### from An he

In [27]:
f = '/home/ahe/ForThePeople/ForStuart/gene_exon_len_ens69.txt'
df = pd.read_csv(f, sep='\t', header=None)
df.head(2)

,0,1
0,ENSG00000000003,2959
1,ENSG00000000005,1603


In [29]:
df[1].sum()

129186872

In [30]:
%%bash
# or use this awk command

cat /home/ahe/ForThePeople/ForStuart/gene_exon_len_ens69.txt|awk '{sum +=$2} END {print sum}'

129186872
